<a href="https://colab.research.google.com/github/sezerblt/data01/blob/master/basic_train_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib
matplotlib.use("Agg")
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", required=True,
	help="goruntuler veri setine giris dizini")
ap.add_argument("-m", "--model", required=True,
	help="Sonuc egitimli modele giden dizin")
ap.add_argument("-l", "--label-bin", required=True,
	help="Sonuc etiketli binarizer giden dizin")
ap.add_argument("-p", "--plot", required=True,
	help=" accuracy/loss oranına giden dizin")
args = vars(ap.parse_args())


print("[INFO] loading images...")
#disteki 
data = []
labels = []
# resimleri al ve karıstır
imagePaths = sorted(list(paths.list_images(args["dataset"])))
random.seed(42)
random.shuffle(imagePaths)

# giris resimleri doongguye  akar
for imagePath in imagePaths:

	#goruntu yukle
	image = cv2.imread(imagePath)
  #en boy boyutlandır
	image = cv2.resize(image, (32, 32)).flatten()
  #listeye ekle
	#sınıf etiketini resim youlundn cıkar ve etikete ekle
	data.append(image)
  #labelx = (imagePath.split(os.path.sep)[-2])
	label = imagePat.split(os.path.sep)[-2]
	labels.append(label)
 
#px degerler 0-1 aralıgına dondur
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# egitim ve test kısımlarını ayırıp teker terker olustur
(trainX, testX, trainY, testY) = train_test_split(data,
											labels, test_size=0.25, random_state=42)

#lbn olustur
lb = LabelBinarizer()
#lb icersindeki essiz etiketleri bul
trainY = lb.fit_transform(trainY)
#test üzerinden gerceklestir
testY = lb.transform(testY)

#Sinir ağımız
model = Sequential()
#3072 (32x32x3) px girisli,1014 dugumlu katman 
model.add(Dense(1024, input_shape=(3072,), activation="sigmoid"))
#512 dugumlu katman
model.add(Dense(512, activation="sigmoid"))
#lb etiketi kadar cikis katmani
model.add(Dense(len(lb.classes_), activation="softmax"))

#ogrenme orani ve egitim periyot süreci(. kez)
INIT_LR = 0.01
EPOCHS = 75

#modeli derleme islemi... 
#ve optimize edici etken:categorical_crossentropy(Binary_crossentropy)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# modeli egitelim
H = model.fit(trainX, trainY, validation_data=(testX, testY),
	epochs=EPOCHS, batch_size=32)
#modelin degerlendirilmesi
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))

# kayıp degeri ve dogrulugunu cizelim
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (Simple NN)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(args["plot"])


python basic_train_network.py --help




